# Visualizing government research and development spending



In [4]:
import pandas as pd
import numpy as np
import plotly.express as px


## Background information

The version of the data that we are using comes from "Tidy Tuesday" ([link](https://github.com/rfordatascience/tidytuesday/tree/master/data/2019/2019-02-12#federal-research-and-development-spending-by-agency)), but the original data source is from the American Association for the Advancement of Science ([link](https://www.aaas.org/programs/r-d-budget-and-policy/historical-trends-federal-rd)). The version of the data we are using has already been cleaned by Tom Mock (a Tidy Tuesday facilitator), and he shares the code he used to clean the data [here](https://github.com/rfordatascience/tidytuesday/tree/master/data/2019/2019-02-12#federal-research-and-development-spending-by-agency).


## Load in the data

First, we load in the three files.

In [11]:
fed_spending = pd.read_csv("data/fed_rd.csv")
energy_spending = pd.read_csv("data/energy_spend.csv")
climate_spending = pd.read_csv("data/climate_spend.csv") 


Let's take a look at the first few rows of each dataset

In [12]:
fed_spending.head()


,department,year,rd_budget,total_outlays,discretionary_outlays,gdp
0,DOD,1976,35696000000,3.718000e+11,1.756000e+11,1790000000000
1,NASA,1976,12513000000,3.718000e+11,1.756000e+11,1790000000000
2,DOE,1976,10882000000,3.718000e+11,1.756000e+11,1790000000000
3,HHS,1976,9226000000,3.718000e+11,1.756000e+11,1790000000000
4,NIH,1976,8025000000,3.718000e+11,1.756000e+11,1790000000000


In [13]:
energy_spending.head()

,department,year,energy_spending
0,Office of Science R&D,1997,3.593300e+09
1,Adv Sci Comp Res*,1997,2.172000e+08
2,Basic Energy Sciences*,1997,9.333000e+08
3,Bio and Env Research*,1997,5.506000e+08
4,Fusion Energy Sciences*,1997,3.314000e+08


In [14]:
climate_spending.head()

,department,year,gcc_spending
0,NASA,2000,1.590942e+09
1,NSF,2000,2.562499e+08
2,Commerce (NOAA),2000,9.181146e+07
3,Energy,2000,1.548462e+08
4,Agriculture,2000,7.673794e+07


## Prepare the data

Next, we want to prepare the data (this is essentially pre-processing the data for our visualization analysis below). To simplify the data, we make the judgment call to aggregate/summarize the spending across all departments for each year.

In [44]:
# aggregate the energy spending by year
energy_by_year = energy_spending.groupby("year")["energy_spending"] \
    .sum() \
    .to_frame() \
    .reset_index()
energy_by_year

,year,energy_spending
0,1997,1.262500e+10
1,1998,1.266250e+10
2,1999,1.365160e+10
3,2000,1.334970e+10
4,2001,1.451120e+10
5,2002,1.471790e+10
6,2003,1.504270e+10
7,2004,1.534310e+10
8,2005,1.471720e+10
9,2006,1.419410e+10


In [45]:
# aggregate the climate spending by year
climate_by_year = climate_spending.groupby("year")["gcc_spending"] \
    .sum() \
    .to_frame("climate_spending") \
    .reset_index()
climate_by_year

,year,climate_spending
0,2000,2.311730e+09
1,2001,2.312562e+09
2,2002,2.195398e+09
3,2003,2.689233e+09
4,2004,2.484252e+09
5,2005,2.283907e+09
6,2006,2.003552e+09
7,2007,2.043967e+09
8,2008,2.069039e+09
9,2009,2.345556e+09


In [66]:
# aggregate the climate spending by year
#fed_spending.groupby("year")["rd_budget"].agg("sum")
fed_spending_by_year = fed_spending[["year","total_outlays", "discretionary_outlays", "gdp"]] \
    .drop_duplicates() \
    .rename(columns = {"total_outlays": "total_spending",
                       "discretionary_outlays": "discretionary_spending"}) \
    .reset_index(drop=True)
fed_spending_by_year

,year,total_spending,discretionary_spending,gdp
0,1976,3.718000e+11,1.756000e+11,1790000000000
1,1977,4.092000e+11,1.971000e+11,2028000000000
2,1978,4.587000e+11,2.187000e+11,2278000000000
3,1979,5.040000e+11,2.400000e+11,2570000000000
4,1980,5.909000e+11,2.763000e+11,2797000000000
5,1981,6.782000e+11,3.079000e+11,3138000000000
6,1982,7.457000e+11,3.259000e+11,3314000000000
7,1983,8.084000e+11,3.533000e+11,3541000000000
8,1984,8.518000e+11,3.794000e+11,3953000000000
9,1985,9.463000e+11,4.158000e+11,4270000000000


In [58]:
rd_budget_by_year = fed_spending.groupby("year")["rd_budget"] \
    .sum() \
    .to_frame("total_rd_budget") \
    .reset_index()
rd_budget_by_year

,year,total_rd_budget
0,1976,86227000000
1,1977,91807000000
2,1978,94864000000
3,1979,96601000000
4,1980,96305000000
5,1981,98304000000
6,1982,95448000000
7,1983,95010000000
8,1984,105371000000
9,1985,114818000000


And then we can combine all three datasets together using `left_join()`, filter to the year 2000 onwards, and scale each of the relevant variables by one million.


In [67]:
govt_spending = fed_spending_by_year.merge(rd_budget_by_year, on="year", how="left") \
    .merge(energy_by_year, on="year", how="left") \
    .merge(climate_by_year, on="year", how="left") \
    .query("year >= 2000") \
    .reset_index(drop=True)
# divide relevant variables by 1_000_000    
spending_var_names = ["gdp", "total_rd_budget", "total_spending", "discretionary_spending", "energy_spending", "climate_spending"]
govt_spending[spending_var_names] = govt_spending[spending_var_names] / 1_000_000
# look at the data
govt_spending

,year,total_spending,discretionary_spending,gdp,total_rd_budget,energy_spending,climate_spending
0,2000,1789000.0,614700.0,10148000.0,142299.0,13349.7,2311.730453
1,2001,1862800.0,649100.0,10565000.0,153197.0,14511.2,2312.561976
2,2002,2010900.0,734000.0,10877000.0,170354.0,14717.9,2195.397900
3,2003,2159900.0,824300.0,11332000.0,192010.0,15042.7,2689.233013
4,2004,2292800.0,895100.0,12089000.0,199104.0,15343.1,2484.251666
5,2005,2472000.0,968500.0,12889000.0,200099.0,14717.2,2283.906691
6,2006,2655000.0,1016700.0,13685000.0,199429.0,14194.1,2003.551973
7,2007,2728700.0,1041600.0,14323000.0,201827.0,14655.6,2043.966543
8,2008,2982500.0,1134800.0,14752000.0,200857.0,15298.2,2069.038746
9,2009,3517700.0,1237500.0,14415000.0,201275.0,16491.6,2345.556400



## Exercise:

Now it's your turn to re-create the figures specified in the exercise.
